# Amazon Review and Metadata Parser

Before we get started to work on the analysis, we want to parse the original dataset to the format we like. The reviews dataset is from [Amazon review data (2018)](http://deepyeti.ucsd.edu/jianmo/amazon/index.html). The dataset is compressed and about 34 GB. But unfortunately we have to decompresse it because the RDD cannnot work on compressed file. The decompressed size is about 126GB.  

The dataset contains a lof of informattion about a review, but in this task we only need the review time, reviewer and product's id. So this process will take these 3 columns and output them to files. 

In [8]:
import json
import datetime
def json_parser(line):
    data = json.loads(line)
    timestamp = data["unixReviewTime"]
    dt = datetime.datetime.fromtimestamp(int(timestamp))
    reviewer = data["reviewerID"]
    product_id = data["asin"]
    review_date =  str(dt.year) + '-' + str(dt.month) 
    return review_date + '\t' + reviewer + '\t' + product_id


In [4]:
text_file = sc.textFile("hdfs://orion11:12001/amazon-reviews/All_Amazon_Review.json")
text_file.map(lambda line: json_parser(line)).saveAsTextFile("hdfs://orion11:12001/amazon-reviews/parsed_data/")


### Metadata
The review's dataset only contains products' id but not products information about what it is. [Amazon review data (2018)](http://deepyeti.ucsd.edu/jianmo/amazon/index.html) also includes the metadata for each products. To make our analysis more comprehensive, the metadata is important.  We will want to parse the metadata in RDD because it is as huge as the review. dataset. The compressed size is 12GB and decompressed size is 110 GB. 


In [9]:
def metadata_parser(line):
    data = json.loads(line)
    asin = ''
    brand = ''
    title = ''
    category = ''
    
    if 'asin' in data:
        asin = data['asin']
        
    if 'brand' in data:
        brand = data['brand']
        
    if 'title' in data:
        title = data['title']
    
    if 'category' in data:
        if len(data['category']) > 0:
            category = data['category'][0]
    
    parsed = {
        'product_id': asin,
        'category': category, 
        'brand' : brand,
        'title' : title
    }
    # asin, category, brand, title, description
    return json.dumps(parsed)

In [11]:
#Modify the port
metadata = sc.textFile("hdfs://orion11:21001/All_Amazon_Meta.json")
metadata.map(lambda line: metadata_parser(line)).saveAsTextFile("hdfs://orion11:21001/parsed-meta")
    